##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reconocimiento de audio sencillo: Reconociendo palabras clave

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/audio/simple_audio"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/audio/simple_audio.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/audio/simple_audio.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver código fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/audio/simple_audio.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a>   </td>
</table>

En este tutorial se muestra cómo preprocesar archivos de audio en formato WAV, además de construir y entrenar un modelo básico de [reconocimiento automático del habla](https://en.wikipedia.org/wiki/Speech_recognition) (ASR) para reconocer diez palabras diferentes. Se utilizará una parte del conjunto de datos [Speech Commands](https://www.tensorflow.org/datasets/catalog/speech_commands) ([Warden, 2018](https://arxiv.org/abs/1804.03209)), que contiene clips de audio cortos (de un segundo o menos) con órdenes, como "abajo", "ir", "izquierda", "no", "derecha", "parar", "arriba" y "sí".

Los sistemas de reconocimiento de voz y audio en el mundo real son complejos. Pero, al igual que [la clasificación de imágenes con el conjunto de datos MNIST](../quickstart/beginner.ipynb), este tutorial debería proporcionarle una comprensión básica de las técnicas implicadas en ellos.

## Preparación

Importa los módulos y dependencias necesarios. Utilizará `tf.keras.utils.audio_dataset_from_directory` (introducido en TensorFlow 2.10), que ayuda a generar conjuntos de datos de clasificación de audio a partir de directorios de archivos `.wav`. También necesitará de [seaborn](https://seaborn.pydata.org) para realizar la visualización en este tutorial.

In [ ]:
!pip install -U -q tensorflow tensorflow_datasets

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display

# Set the seed value for experiment reproducibility.
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

## Importar el conjunto de datos del mini comandos de voz

Para ahorrar tiempo al cargar datos, trabajará con una versión más pequeña del conjunto de datos Speech Commands. El conjunto de datos [original](https://www.tensorflow.org/datasets/catalog/speech_commands) consta de más de 105,000 archivos de audio en formato [WAV (Waveform)](https://www.aelius.com/njh/wavemetatools/doc/riffmci.pdf) de personas diciendo 35 palabras diferentes. Estos datos fueron recopilados por Google y publicados bajo una licencia CC BY.

Descargue y extraiga el archivo `mini_speech_commands.zip` que contiene los conjuntos de datos de Speech Commands más pequeños con `tf.keras.utils.get_file`:

In [ ]:
DATASET_PATH = 'data/mini_speech_commands'

data_dir = pathlib.Path(DATASET_PATH)
if not data_dir.exists():
  tf.keras.utils.get_file(
      'mini_speech_commands.zip',
      origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
      extract=True,
      cache_dir='.', cache_subdir='data')

Los clips de audio del conjunto de datos se almacenan en ocho carpetas correspondientes a cada comando de voz: `no`, `yes`, `down`, `go`, `left`, `up`, `right` y `stop`:

In [ ]:
commands = np.array(tf.io.gfile.listdir(str(data_dir)))
commands = commands[(commands != 'README.md') & (commands != '.DS_Store')]
print('Commands:', commands)

Dividido en directorios de esta manera, puede cargar fácilmente los datos utilizando `keras.utils.audio_dataset_from_directory`.

Los clips de audio son de 1 segundo o menos a 16kHz. El `output_sequence_length=16000` rellena los cortos a exactamente 1 segundo (y recortaría los más largos) para que puedan ser fácilmente agrupados.

In [ ]:
train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory=data_dir,
    batch_size=64,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both')

label_names = np.array(train_ds.class_names)
print()
print("label names:", label_names)

El conjunto de datos contiene ahora lotes de clips de audio y etiquetas con números enteros. Los clips de audio tienen una forma de `(batch, samples, channels)`. 

In [ ]:
train_ds.element_spec

Este conjunto de datos sólo contiene un audio monocanal, así que utilice la función `tf.squeeze` para eliminar el eje adicional:

In [ ]:
def squeeze(audio, labels):
  audio = tf.squeeze(audio, axis=-1)
  return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

La función `utils.audio_dataset_from_directory` sólo devuelve hasta dos divisiones. Es una buena idea mantener un conjunto de prueba separado de su conjunto de validación. Lo ideal sería mantenerlo en un directorio separado, pero en este caso puede utilizar `Dataset.shard` para dividir el conjunto de validación en dos mitades. Tenga en cuenta que iterar sobre **cualquier** fragmento cargará **todos** los datos, y sólo mantendrá su fracción. 

In [ ]:
test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)

In [ ]:
for example_audio, example_labels in train_ds.take(1):  
  print(example_audio.shape)
  print(example_labels.shape)

Vamos a graficar algunas ondas de audio:

In [ ]:
label_names[[1,1,3,0]]

In [ ]:
plt.figure(figsize=(16, 10))
rows = 3
cols = 3
n = rows * cols
for i in range(n):
  plt.subplot(rows, cols, i+1)
  audio_signal = example_audio[i]
  plt.plot(audio_signal)
  plt.title(label_names[example_labels[i]])
  plt.yticks(np.arange(-1.2, 1.2, 0.2))
  plt.ylim([-1.1, 1.1])

## Convertir ondas en espectrogramas

Las ondas del conjunto de datos se representan en el dominio del tiempo. A continuación, transformaremos las ondas de las señales en el dominio del tiempo en señales en el dominio de la frecuencia del tiempo calculando la [transformación de Fourier en tiempo corto (STFT)](https://en.wikipedia.org/wiki/Short-time_Fourier_transform) para convertir las formas de onda en [espectrogramas](https://en.wikipedia.org/wiki/Spectrogram), que muestran los cambios de frecuencia en el tiempo y pueden representarse como imágenes 2D. Las imágenes del espectrograma se introducirán en la red neuronal para entrenar el modelo.

Una transformada de Fourier (`tf.signal.fft`) convierte una señal en sus frecuencias componentes, pero pierde toda la información temporal. A su vez, la STFT (`tf.signal.stft`) divide la señal en ventanas de tiempo y ejecuta una transformada de Fourier en cada ventana, conservando parte de la información temporal y devolviendo un tensor 2D sobre el que se pueden ejecutar convoluciones estándar.

Crear una función de utilidad para convertir las ondas en espectrogramas:

- Las ondas deben tener la misma longitud, para que cuando las convierta en espectrogramas, los resultados tengan dimensiones similares. Para ello, basta con poner amortiguado en cero los clips de audio de menos de un segundo (con `tf.zeros`).
- Cuando llame a `tf.signal.stft`, elija los parámetros `frame_length` y `frame_step` de forma que la "imagen" del espectrograma generado sea casi cuadrada. Para obtener más información sobre la elección de los parámetros STFT, consulte [este video de Coursera](https://www.coursera.org/lecture/audio-signal-processing/stft-2-tjEQe) sobre el procesamiento de señales de audio y STFT.
- La STFT produce un arreglo de números complejos que representan la magnitud y la fase. Sin embargo, en este tutorial sólo se utilizará la magnitud, que se puede derivar mediante la aplicación de `tf.abs` en la salida de `tf.signal.stft`.

In [ ]:
def get_spectrogram(waveform):
  # Convert the waveform to a spectrogram via a STFT.
  spectrogram = tf.signal.stft(
      waveform, frame_length=255, frame_step=128)
  # Obtain the magnitude of the STFT.
  spectrogram = tf.abs(spectrogram)
  # Add a `channels` dimension, so that the spectrogram can be used
  # as image-like input data with convolution layers (which expect
  # shape (`batch_size`, `height`, `width`, `channels`).
  spectrogram = spectrogram[..., tf.newaxis]
  return spectrogram

A continuación, empieza a explorar los datos. Imprime las formas de la onda tensorizada de un ejemplo y el espectrograma correspondiente, y reproduce el audio original:

In [ ]:
for i in range(3):
  label = label_names[example_labels[i]]
  waveform = example_audio[i]
  spectrogram = get_spectrogram(waveform)

  print('Label:', label)
  print('Waveform shape:', waveform.shape)
  print('Spectrogram shape:', spectrogram.shape)
  print('Audio playback')
  display.display(display.Audio(waveform, rate=16000))

Ahora, defina una función para visualizar un espectrograma:

In [ ]:
def plot_spectrogram(spectrogram, ax):
  if len(spectrogram.shape) > 2:
    assert len(spectrogram.shape) == 3
    spectrogram = np.squeeze(spectrogram, axis=-1)
  # Convert the frequencies to log scale and transpose, so that the time is
  # represented on the x-axis (columns).
  # Add an epsilon to avoid taking a log of zero.
  log_spec = np.log(spectrogram.T + np.finfo(float).eps)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)

Grafique la onda del ejemplo a lo largo del tiempo y el espectrograma correspondiente (frecuencias a lo largo del tiempo):

In [ ]:
fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(waveform.shape[0])
axes[0].plot(timescale, waveform.numpy())
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 16000])

plot_spectrogram(spectrogram.numpy(), axes[1])
axes[1].set_title('Spectrogram')
plt.suptitle(label.title())
plt.show()

Ahora, cree conjuntos de datos de espectrogramas a partir de los conjuntos de datos de audio:

In [ ]:
def make_spec_ds(ds):
  return ds.map(
      map_func=lambda audio,label: (get_spectrogram(audio), label),
      num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
train_spectrogram_ds = make_spec_ds(train_ds)
val_spectrogram_ds = make_spec_ds(val_ds)
test_spectrogram_ds = make_spec_ds(test_ds)

Examine los espectrogramas de distintos ejemplos del conjunto de datos:

In [ ]:
for example_spectrograms, example_spect_labels in train_spectrogram_ds.take(1):
  break

In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(16, 9))

for i in range(n):
    r = i // cols
    c = i % cols
    ax = axes[r][c]
    plot_spectrogram(example_spectrograms[i].numpy(), ax)
    ax.set_title(label_names[example_spect_labels[i].numpy()])

plt.show()

## Construya y entrene el modelo.

Agregue las operaciones `Dataset.cache` y `Dataset.prefetch` para reducir la latencia en la lectura mientras se entrena el modelo:

In [ ]:
train_spectrogram_ds = train_spectrogram_ds.cache().shuffle(10000).prefetch(tf.data.AUTOTUNE)
val_spectrogram_ds = val_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)
test_spectrogram_ds = test_spectrogram_ds.cache().prefetch(tf.data.AUTOTUNE)

Para el modelo, utilizará una red neuronal convolucional (CNN) sencilla, ya que transformó los archivos de audio en imágenes de espectrogramas.

Su modelo `tf.keras.Sequential` utilizará las siguientes capas de preprocesamiento de Keras:

- `tf.keras.layers.Resizing`: para reducir la muestra de entrada y permitir que el modelo se entrene más rápido.
- `tf.keras.layers.Normalization`: para normalizar cada píxel de la imagen basado en su media y desviación estándar.

Para la capa `Normalization`, su método `adapt` necesitaría primero recibir la llamada de los datos de entrenamiento para calcular las estadísticas agregadas (es decir, la media y la desviación estándar).

In [ ]:
input_shape = example_spectrograms.shape[1:]
print('Input shape:', input_shape)
num_labels = len(label_names)

# Instantiate the `tf.keras.layers.Normalization` layer.
norm_layer = layers.Normalization()
# Fit the state of the layer to the spectrograms
# with `Normalization.adapt`.
norm_layer.adapt(data=train_spectrogram_ds.map(map_func=lambda spec, label: spec))

model = models.Sequential([
    layers.Input(shape=input_shape),
    # Downsample the input.
    layers.Resizing(32, 32),
    # Normalize.
    norm_layer,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

model.summary()

Configure el modelo Keras con el optimizador Adam y la pérdida de entropía cruzada:

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

Entrene el modelo durante 10 épocas con fines demostrativos:

In [ ]:
EPOCHS = 10
history = model.fit(
    train_spectrogram_ds,
    validation_data=val_spectrogram_ds,
    epochs=EPOCHS,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

Vamos a graficar las curvas de pérdidas de entrenamiento y validación para revisar cómo ha mejorado su modelo durante el entrenamiento:

In [ ]:
metrics = history.history
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch')
plt.ylabel('Loss [CrossEntropy]')

plt.subplot(1,2,2)
plt.plot(history.epoch, 100*np.array(metrics['accuracy']), 100*np.array(metrics['val_accuracy']))
plt.legend(['accuracy', 'val_accuracy'])
plt.ylim([0, 100])
plt.xlabel('Epoch')
plt.ylabel('Accuracy [%]')

## Evaluar el rendimiento del modelo

Ejecute el modelo en el conjunto de pruebas y verifique el rendimiento del modelo:

In [ ]:
model.evaluate(test_spectrogram_ds, return_dict=True)

### Visualizar una matriz de confusión

Utilice una [matriz de confusión](https://developers.google.com/machine-learning/glossary#confusion-matrix) para verificar que el modelo clasificó bien cada uno de los comandos del conjunto de prueba:


In [ ]:
y_pred = model.predict(test_spectrogram_ds)

In [ ]:
y_pred = tf.argmax(y_pred, axis=1)

In [ ]:
y_true = tf.concat(list(test_spectrogram_ds.map(lambda s,lab: lab)), axis=0)

In [ ]:
confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=label_names,
            yticklabels=label_names,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show()

## Ejecutar la inferencia en un archivo de audio

Por último, verifique la salida de las predicciones del modelo utilizando un archivo de audio de entrada en el que alguien diga "no". ¿Cómo funciona su modelo?

In [ ]:
x = data_dir/'no/01bb6a2a_nohash_0.wav'
x = tf.io.read_file(str(x))
x, sample_rate = tf.audio.decode_wav(x, desired_channels=1, desired_samples=16000,)
x = tf.squeeze(x, axis=-1)
waveform = x
x = get_spectrogram(x)
x = x[tf.newaxis,...]

prediction = model(x)
x_labels = ['no', 'yes', 'down', 'go', 'left', 'up', 'right', 'stop']
plt.bar(x_labels, tf.nn.softmax(prediction[0]))
plt.title('No')
plt.show()

display.display(display.Audio(waveform, rate=16000))

Como sugiere la salida, su modelo debería haber reconocido el comando de audio como "no".

## Exportar el modelo con procesamiento previo

El modelo no es muy fácil de usar si tiene que aplicar esos pasos de procesamiento previo antes de pasar los datos al modelo para que realice inferencias. De modo que debe construir una versión de extremo a extremo:

In [ ]:
class ExportModel(tf.Module):
  def __init__(self, model):
    self.model = model

    # Accept either a string-filename or a batch of waveforms.
    # YOu could add additional signatures for a single wave, or a ragged-batch. 
    self.__call__.get_concrete_function(
        x=tf.TensorSpec(shape=(), dtype=tf.string))
    self.__call__.get_concrete_function(
       x=tf.TensorSpec(shape=[None, 16000], dtype=tf.float32))


  @tf.function
  def __call__(self, x):
    # If they pass a string, load the file and decode it. 
    if x.dtype == tf.string:
      x = tf.io.read_file(x)
      x, _ = tf.audio.decode_wav(x, desired_channels=1, desired_samples=16000,)
      x = tf.squeeze(x, axis=-1)
      x = x[tf.newaxis, :]
    
    x = get_spectrogram(x)  
    result = self.model(x, training=False)
    
    class_ids = tf.argmax(result, axis=-1)
    class_names = tf.gather(label_names, class_ids)
    return {'predictions':result,
            'class_ids': class_ids,
            'class_names': class_names}

Pruebe el modelo "exportar":

In [ ]:
export = ExportModel(model)
export(tf.constant(str(data_dir/'no/01bb6a2a_nohash_0.wav')))

Guarde y vuelva a cargar el modelo, el modelo que se cargó de nuevo da un resultado idéntico:

In [ ]:
tf.saved_model.save(export, "saved")
imported = tf.saved_model.load("saved")
imported(waveform[tf.newaxis, :])

## Siguientes pasos

Este tutorial muestra cómo llevar a cabo una sencilla clasificación de audio/reconocimiento automático del habla utilizando una red neuronal convolucional con TensorFlow y Python. Para obtener más información, tenga en cuenta los siguientes recursos:

- En el tutorial [Clasificación de sonido con YAMNet](https://www.tensorflow.org/hub/tutorials/yamnet) se muestra cómo utilizar el aprendizaje por transferencia para realizar una clasificación de audio.
- Los blocs de notas del [Desafío de reconocimiento del habla con TensorFlow de Kaggle](https://www.kaggle.com/c/tensorflow-speech-recognition-challenge/overview).
- El [TensorFlow.js - Reconocimiento de audio mediante aprendizaje por transferencia codelab](https://codelabs.developers.google.com/codelabs/tensorflowjs-audio-codelab/index.html#0) le enseña a construir su propia aplicación web interactiva para realizar clasificaciones de audio.
- [Un tutorial sobre deep learning para la recuperación de información musical](https://arxiv.org/abs/1709.04396) (Choi et al., 2017) en arXiv.
- TensorFlow también tiene soporte adicional para la [preparación y aumento de datos de audio](https://www.tensorflow.org/io/tutorials/audio) que le ayudará con sus propios proyectos basados en audio.
- Considere la posibilidad de utilizar la biblioteca [librosa](https://librosa.org/) para realizar análisis de música y audio.